# Mediator Method Design Pattern:

## Some Use Cases:
1. Centralized Control of Data Flows in ETL Pipelines:
- Use Case: In complex ETL (Extract, Transform, Load) workflows where multiple systems (data sources, processors, and sinks) interact with each other, the Mediator pattern can manage communication and coordination between these systems.
- Benefit: The Mediator ensures that all systems interact through a central mediator, simplifying the communication and reducing the direct dependencies between them, making the entire pipeline easier to manage.
2. Coordinating Data Access Across Microservices
- Use Case: In a microservices architecture, where various services require access to shared data (e.g., user profiles, transactions), the Mediator pattern can centralize the management of requests, ensuring consistency and preventing data conflicts.
- Benefit: The Mediator enables communication between different services through a single access point, decoupling services and reducing direct dependencies, which is essential for scaling and maintaining consistency across the system.
3. Managing Complex Data Integration Tasks
- Use Case: When integrating data from multiple heterogeneous sources (e.g., databases, APIs, third-party systems), the Mediator can facilitate and streamline the interaction between these sources by centralizing the logic of how data is exchanged.
- Benefit: The Mediator simplifies the integration process, ensuring that each system only communicates with the mediator, rather than managing multiple direct dependencies, thus making the integration more flexible and maintainable.

## 1. Scenario: Mediator Design Pattern
- In a chat application, multiple users communicate in a group chat. Instead of each user directly managing their interactions with others, a central "ChatRoom" (mediator) handles the communication.

### 1.1 Using Traditional Method:

In [1]:
class User:
    def __init__(self, name):
        self.name = name

    def send_message(self, receiver, message):
        print(f"{self.name} to {receiver.name}: {message}")


# Usage
user1 = User("Alice")
user2 = User("Bob")
user3 = User("Charlie")

user1.send_message(user2, "Hi Bob!")  # Alice to Bob: Hi Bob!
user2.send_message(user3, "Hello Charlie!")  # Bob to Charlie: Hello Charlie!


Alice to Bob: Hi Bob!
Bob to Charlie: Hello Charlie!


### Issues with Traditional Method:
- Tightly Coupled: Users interact directly with each other, making the system hard to modify.
- Scalability Problems: Adding new users increases complexity due to direct communication links.
- Difficult Coordination: Managing group interactions becomes cumbersome.

### 1.2 Using Mediator Method Pattern:

### Components of Mediator Pattern:
- Mediator Interface: Defines the contract for communication between objects.
- Concrete Mediator: Implements the communication logic, coordinating interactions.
- Colleagues: Abstract base class for objects that interact through the mediator.
- Concrete Colleagues: Specific implementations of Colleagues that communicate via the mediator

In [3]:
# Mediator Interface
class ChatRoomMediator:
    def show_message(self, user, message):
        pass


# Concrete Mediator
class ChatRoom(ChatRoomMediator):
    def show_message(self, user, message):
        print(f"[{user.name}]: {message}")


# Colleague Class
class User:
    def __init__(self, name, chatroom):
        self.name = name
        self.chatroom = chatroom

    def send_message(self, message):
        self.chatroom.show_message(self, message)


# Usage
chatroom = ChatRoom()
user1 = User("Alice", chatroom)
user2 = User("Bob", chatroom)
user3 = User("Charlie", chatroom)

user1.send_message("Hi everyone!")  # [Alice]: Hi everyone!
user2.send_message("Hello Alice!")  # [Bob]: Hello Alice!
user3.send_message("Hey all!")      # [Charlie]: Hey all!


[Alice]: Hi everyone!
[Bob]: Hello Alice!
[Charlie]: Hey all!


### How Mediator Solves the Problem
- Loose Coupling: Users interact with the mediator, not directly with each other.
- Simplified Communication: A central mediator manages all communication logic.
- Scalability: Adding or modifying users doesn’t affect the existing system.
- Centralized Control: The mediator provides a single point of communication, making group interactions efficient.

## 2. Scenario: Data Pipeline Coordination
- In a data engineering context, consider a data pipeline where different components (like data ingestion, data transformation, and data storage) need to work together seamlessly. Each component depends on the successful execution of others, and direct communication between them can lead to tight coupling, making the pipeline difficult to maintain or modify.

- Using the Mediator design pattern, we can introduce a central mediator to coordinate these interactions without the components needing to know about each other directly.

In [4]:
# Mediator Interface
class PipelineMediator:
    def notify(self, sender, event):
        pass

# Concrete Mediator
class DataPipelineMediator(PipelineMediator):
    def __init__(self):
        self.ingest = None
        self.transform = None
        self.store = None

    def notify(self, sender, event):
        if event == "data_ingested":
            print("Mediator: Data ingested. Triggering transformation...")
            self.transform.process_data(sender.data)
        elif event == "data_transformed":
            print("Mediator: Data transformed. Triggering storage...")
            self.store.save_data(sender.data)

# Abstract Colleague
class PipelineComponent:
    def __init__(self, mediator):
        self.mediator = mediator

# Concrete Colleagues
class DataIngest(PipelineComponent):
    def __init__(self, mediator):
        super().__init__(mediator)
        self.data = None

    def ingest_data(self, data):
        print(f"Ingesting data: {data}")
        self.data = data
        self.mediator.notify(self, "data_ingested")

class DataTransform(PipelineComponent):
    def __init__(self, mediator):
        super().__init__(mediator)
        self.data = None

    def process_data(self, data):
        print(f"Transforming data: {data}")
        self.data = data.upper()  # Example transformation
        self.mediator.notify(self, "data_transformed")

class DataStore(PipelineComponent):
    def __init__(self, mediator):
        super().__init__(mediator)

    def save_data(self, data):
        print(f"Storing data: {data}")

# Usage
mediator = DataPipelineMediator()
ingest = DataIngest(mediator)
transform = DataTransform(mediator)
store = DataStore(mediator)

mediator.ingest = ingest
mediator.transform = transform
mediator.store = store

# Start the pipeline
ingest.ingest_data("raw_data")


Ingesting data: raw_data
Mediator: Data ingested. Triggering transformation...
Transforming data: raw_data
Mediator: Data transformed. Triggering storage...
Storing data: RAW_DATA


### Advantage of using Mediator Pattern:
- Decoupling: Components (ingest, transform, store) do not need to know about each other. All communication is handled by the mediator.
- Scalability: Adding new pipeline stages or modifying existing ones can be done by updating the mediator, without changing the other components.
- Centralized Control: The mediator manages the workflow, making it easier to debug and extend.